In [5]:
!pip install seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
sales_train = pd.read_csv('sales_train.csv')
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [7]:
    grid = []
    for block_num in sales_train['date_block_num'].unique():
        cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales_train[sales_train['date_block_num']==block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
    grid

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0
...,...,...,...
10913845,21,7635,33
10913846,21,7638,33
10913847,21,7640,33
10913848,21,7632,33


In [8]:
sales_train['item_cnt_day'] = sales_train['item_cnt_day']
sales_train['item_cnt_day']

0          1.0
1          1.0
2         -1.0
3          1.0
4          1.0
          ... 
2935844    1.0
2935845    1.0
2935846    1.0
2935847    1.0
2935848    1.0
Name: item_cnt_day, Length: 2935849, dtype: float64

In [9]:
groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])
trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
trainset = trainset.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
trainset['item_cnt_month'] = trainset['item_cnt_month']
trainset = pd.merge(grid,trainset,how='left',on=index_cols)
trainset.item_cnt_month = trainset.item_cnt_month.fillna(0)
trainset.drop('item_price',axis=1)

,shop_id,item_id,date_block_num,item_cnt_month
0,59,22154,0,1.0
1,59,2552,0,0.0
2,59,2554,0,0.0
3,59,2555,0,0.0
4,59,2564,0,0.0
...,...,...,...,...
10913845,21,7635,33,0.0
10913846,21,7638,33,0.0
10913847,21,7640,33,0.0
10913848,21,7632,33,0.0


In [10]:
testset = pd.read_csv('test.csv')
testset.drop('ID',axis=1,inplace=True)
train_X=trainset[index_cols]
train_Y=trainset['item_cnt_month']
testset['date_block_num']=34
testset

,shop_id,item_id,date_block_num
0,5,5037,34
1,5,5320,34
2,5,5233,34
3,5,5232,34
4,5,5268,34
...,...,...,...
214195,45,18454,34
214196,45,16188,34
214197,45,15757,34
214198,45,19648,34


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
lm = LinearRegression()
lm.fit(train_X,train_Y)

LinearRegression()

In [13]:
predictions = lm.predict(testset)

In [16]:
df = pd.DataFrame(predictions,columns=['item_cnt_month'])
df['ID'] = df.index
df = df.set_index('ID')
df.to_csv('Answer.csv')

In [17]:
pd.read_csv('Answer.csv')

,ID,item_cnt_month
0,0,0.314955
1,1,0.312767
2,2,0.313440
3,3,0.313447
4,4,0.313169
...,...,...
214195,214195,0.284939
214196,214196,0.302458
214197,214197,0.305790
214198,214198,0.275708
